# Readme

This file includes:

- data formatting according to each model's specifications
- generation of 15 random trial partitions for cross-validation
- construction of the datasets corresponding to the scenarios defined in the article

# Setup

In [1]:
library ("ggplot2")
library ("dplyr") 
library ("tibble")
library ("tidyr")
library ("kableExtra")
library ("jsonlite")

source (file = "../../01_functions/functions.R")

load (file = "processed_data/data.Rdata")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows




In [2]:
data = data %>% select (- c (eid, exper, field))

# Data for static random forest

In [3]:
head (data) %>% embed

e.cum,e.rel,j.NH3,pmid,meas.tech,country,inst,ct,dt,air.temp,wind.2m,rain.rate,tan.app,app.rate,man.dm,man.ph,man.source,t.incorp,app.mthd,incorp
7.1480,0.0585374,1.7870000,182,micro met,DK,104,4.00,4.00,8.20,8.10,0.0000000,122.11,31.8,3.7,7.35,pig,1000,bc,none
8.2921,0.0679068,0.0673000,182,micro met,DK,104,21.00,17.00,4.45,3.98,0.0000000,122.11,31.8,3.7,7.35,pig,1000,bc,none
11.8310,0.0968881,0.1490063,182,micro met,DK,104,44.75,23.75,7.22,6.57,0.0084211,122.11,31.8,3.7,7.35,pig,1000,bc,none
12.6320,0.1034477,0.0166045,182,micro met,DK,104,92.99,48.24,10.04,4.86,0.0559700,122.11,31.8,3.7,7.35,pig,1000,bc,none
16.4190,0.1344607,0.0525972,182,micro met,DK,104,164.99,72.00,12.13,5.38,0.0888890,122.11,31.8,3.7,7.35,pig,1000,bc,none
3.4692,0.0594856,0.5782000,183,micro met,DK,104,6.00,6.00,13.77,4.31,0.0000000,58.32,21.6,2.8,7.71,pig,1000,bsth,none


In [4]:
# Interpolation of meteorological data at a time step of 1 for each pmid, and creation of the 'group' variable,
# which corresponds to the 6 time intervals [0, 4], [4, 8], ..., [16, 20], [20, trial duration].

pmids = data$pmid %>% unique

DT = 1

interpolated_data_meteo = NULL

for (i in c (1 : length (pmids))){
    
    data_tmp = data %>% filter (pmid == pmids[i]) 
    
    max_dt = max (data_tmp$ct)

    seq_time = seq (1, max_dt - 1e-8 + DT, by = DT)

    group = c (
        sapply (c (1 : 5), function (j) rep (j, 4)) %>% as.vector,
        rep (6, (length (seq_time) - 20))
    )


    app.air.temp = approx (data_tmp$ct, data_tmp$air.temp, xout = seq_time, rule = 2)
    app.wind = approx (data_tmp$ct, data_tmp$wind.2m, xout = seq_time, rule = 2)
    app.rain = approx (data_tmp$ct, data_tmp$rain.rate, xout = seq_time, rule = 2)
    
    data_tmp = data.frame (
        pmid = data_tmp$pmid[1],
        air.temp = app.air.temp$y,
        wind.2m = app.wind$y,
        rain.rate = app.rain$y,
        ct = seq_time,
        group = group
    )
    
    interpolated_data_meteo = rbind (interpolated_data_meteo, data_tmp)
    
}

In [5]:
interpolated_data_meteo %>% head

,pmid,air.temp,wind.2m,rain.rate,ct,group
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,182,8.200000,8.100000,0,1,1
2,182,8.200000,8.100000,0,2,1
3,182,8.200000,8.100000,0,3,1
4,182,8.200000,8.100000,0,4,1
5,182,7.979412,7.857647,0,5,2
6,182,7.758824,7.615294,0,6,2


In [6]:
# Averaging meteorological variables by pmid and group, and transforming them into class variables

data_meteo = interpolated_data_meteo %>%

    summarise (
        air_temp = mean (air.temp),
        wind.2m = mean (wind.2m), 
        rain.rate = mean (rain.rate),
        .by = c (pmid, group)
    ) %>% 

    pivot_wider (names_from = group, values_from = c (air_temp, wind.2m, rain.rate))

In [7]:
data_random_forest_statique = data %>%
    filter (ct == max (ct), .by = pmid) %>%
    select (- j.NH3, - air.temp, - wind.2m, - rain.rate, - dt) %>%
    left_join (data_meteo, by = "pmid")

In [8]:
dim (data_random_forest_statique)

[1] 699  33

In [9]:
data_random_forest_statique %>% head %>% embed

e.cum,e.rel,pmid,meas.tech,country,inst,ct,tan.app,app.rate,man.dm,man.ph,man.source,t.incorp,app.mthd,incorp,air_temp_1,air_temp_2,air_temp_3,air_temp_4,air_temp_5,air_temp_6,wind.2m_1,wind.2m_2,wind.2m_3,wind.2m_4,wind.2m_5,wind.2m_6,rain.rate_1,rain.rate_2,rain.rate_3,rain.rate_4,rain.rate_5,rain.rate_6
16.4190,0.1344607,182,micro met,DK,104,164.99,122.110,31.8,3.7,7.35,pig,1000,bc,none,8.20000,7.648529,6.766177,5.883824,5.001471,9.38909,8.100000,7.494118,6.524706,5.555294,4.585882,5.340230,0,0,0,0,0,0.0476788
6.9332,0.1188820,183,micro met,DK,104,163.20,58.320,21.6,2.8,7.71,pig,1000,bsth,none,13.77000,13.568793,12.562759,11.489655,10.416552,10.00659,4.310000,4.261897,4.021379,3.764828,3.508276,4.141415,0,0,0,0,0,0.0000000
12.9650,0.1647730,184,micro met,DK,104,164.15,78.684,24.9,3.4,7.61,pig,1000,bc,none,11.15000,10.898400,10.179867,9.454533,8.729200,11.11668,8.270000,8.081300,7.542400,6.998400,6.454400,5.481192,0,0,0,0,0,0.0025087
8.2469,0.1310696,185,micro met,DK,104,165.00,62.920,24.2,2.7,7.58,pig,1000,bsth,none,11.62000,11.509310,10.734483,9.848966,8.963448,11.08267,8.130000,8.065517,7.614138,7.098276,6.582414,5.502466,0,0,0,0,0,0.0000000
11.0830,0.1942103,186,micro met,DK,104,165.85,57.067,38.3,5.2,6.70,pig,1000,bc,none,17.40353,16.274401,14.834880,13.395359,11.972193,11.95045,3.685621,3.473323,3.202665,2.932006,2.666729,5.014117,0,0,0,0,0,0.0516128
8.5522,0.1833780,187,micro met,DK,104,166.25,46.637,31.3,5.2,6.70,pig,1000,bsth,none,16.98000,16.322090,15.006269,13.690448,12.374627,11.93442,3.640000,3.512239,3.256716,3.001194,2.745672,4.989700,0,0,0,0,0,0.0521255


In [10]:
save (data_random_forest_statique, file = "processed_data/data_random_forest_statique.Rdata")

# Data for dynamic random forest

In [11]:
# Interpolation of meteorological variables and cumulative ammonia emissions at different time steps

pmids = data$pmid %>% unique

DT = c (2, 4, 6, 8, 10)

data_random_forest_dynamique = NULL

for (i in c (1 : length (pmids))){

    for (dt in DT) {
    
        data_tmp = data %>% filter (pmid == pmids[i]) 
    
        data_tmp = rbind (data_tmp [1, ], data_tmp)
        data_tmp$e.cum[1] = 0
        data_tmp$ct[1] = 0
        
        max_dt = max (data_tmp$ct)
    
        seq_time = seq (0, max_dt - 1e-8 + dt, by = dt)
    
        app.e.cum = approx (data_tmp$ct, data_tmp$e.cum, xout = seq_time, rule = 2) 
        app.air.temp = approx (data_tmp$ct, data_tmp$air.temp, xout = seq_time, rule = 2)
        app.wind = approx (data_tmp$ct, data_tmp$wind.2m, xout = seq_time, rule = 2)
        app.rain = approx (data_tmp$ct, data_tmp$rain.rate, xout = seq_time, rule = 2)
        
        data_tmp = data.frame (
            e.cum = app.e.cum$y,
            dt = dt,
            inst = data_tmp$inst[1],
            pmid = data_tmp$pmid[1],
            country = data_tmp$country[1],
            meas.tech = data_tmp$meas.tech[1],
            ct = seq_time,
            air.temp = app.air.temp$y,
            wind.2m = app.wind$y,
            rain.rate = app.rain$y,
            tan.app = data_tmp$tan.app[1],
            app.mthd = data_tmp$app.mthd[1],
            app.rate = data_tmp$app.rate[1],
            man.dm = data_tmp$man.dm[1],
            man.ph = data_tmp$man.ph[1],
            man.source = data_tmp$man.source[1],
            incorp = data_tmp$incorp[1],
            t.incorp = data_tmp$t.incorp[1]
        )
    
        data_tmp = data_tmp [- 1, ]
        data_tmp$e.cum_shift = c (0, data_tmp$e.cum [c (1 : (nrow (data_tmp) - 1))])
    
        data_tmp = data_tmp %>% relocate (e.cum_shift, .after = e.cum)
        
        data_random_forest_dynamique = rbind (data_random_forest_dynamique, data_tmp)
    }
    
}

In [12]:
data_random_forest_dynamique %>% head (n = 2) %>% embed

,e.cum,e.cum_shift,dt,inst,pmid,country,meas.tech,ct,air.temp,wind.2m,rain.rate,tan.app,app.mthd,app.rate,man.dm,man.ph,man.source,incorp,t.incorp
2,3.574,0.000,2,104,182,DK,micro met,2,8.2,8.1,0,122.11,bc,31.8,3.7,7.35,pig,none,1000
3,7.148,3.574,2,104,182,DK,micro met,4,8.2,8.1,0,122.11,bc,31.8,3.7,7.35,pig,none,1000


In [13]:
save (data_random_forest_dynamique, file = "processed_data/data_random_forest_dynamique.Rdata")

# Data for static neural networks

## Without embeddings

In [14]:
data_random_forest_statique %>% head (n = 2) %>% embed

e.cum,e.rel,pmid,meas.tech,country,inst,ct,tan.app,app.rate,man.dm,man.ph,man.source,t.incorp,app.mthd,incorp,air_temp_1,air_temp_2,air_temp_3,air_temp_4,air_temp_5,air_temp_6,wind.2m_1,wind.2m_2,wind.2m_3,wind.2m_4,wind.2m_5,wind.2m_6,rain.rate_1,rain.rate_2,rain.rate_3,rain.rate_4,rain.rate_5,rain.rate_6
16.4190,0.1344607,182,micro met,DK,104,164.99,122.11,31.8,3.7,7.35,pig,1000,bc,none,8.20,7.648529,6.766177,5.883824,5.001471,9.38909,8.10,7.494118,6.524706,5.555294,4.585882,5.340230,0,0,0,0,0,0.0476788
6.9332,0.1188820,183,micro met,DK,104,163.20,58.32,21.6,2.8,7.71,pig,1000,bsth,none,13.77,13.568793,12.562759,11.489655,10.416552,10.00659,4.31,4.261897,4.021379,3.764828,3.508276,4.141415,0,0,0,0,0,0.0000000


In [15]:
dim (data_random_forest_statique)

[1] 699  33

In [16]:
data_static_nn_1 = data_random_forest_statique %>%

    mutate (app.mthd_bc = ifelse (app.mthd == "bc", 1, 0)) %>%
    mutate (app.mthd_bsth = ifelse (app.mthd == "bsth", 1, 0)) %>%
    mutate (app.mthd_ts = ifelse (app.mthd == "ts", 1, 0)) %>%
    mutate (app.mthd_os = ifelse (app.mthd == "os", 1, 0)) %>%

    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%

    mutate (incorp_none = ifelse (incorp == "none", 1, 0)) %>%
    mutate (incorp_shallow = ifelse (incorp == "shallow", 1, 0)) %>%

    select (- app.mthd, - incorp)

In [17]:
head (data_static_nn_1, n = 2) %>% embed

e.cum,e.rel,pmid,meas.tech,country,inst,ct,tan.app,app.rate,man.dm,man.ph,man.source,t.incorp,air_temp_1,air_temp_2,air_temp_3,air_temp_4,air_temp_5,air_temp_6,wind.2m_1,wind.2m_2,wind.2m_3,wind.2m_4,wind.2m_5,wind.2m_6,rain.rate_1,rain.rate_2,rain.rate_3,rain.rate_4,rain.rate_5,rain.rate_6,app.mthd_bc,app.mthd_bsth,app.mthd_ts,app.mthd_os,incorp_none,incorp_shallow
16.4190,0.1344607,182,micro met,DK,104,164.99,122.11,31.8,3.7,7.35,0,1000,8.20,7.648529,6.766177,5.883824,5.001471,9.38909,8.10,7.494118,6.524706,5.555294,4.585882,5.340230,0,0,0,0,0,0.0476788,1,0,0,0,1,0
6.9332,0.1188820,183,micro met,DK,104,163.20,58.32,21.6,2.8,7.71,0,1000,13.77,13.568793,12.562759,11.489655,10.416552,10.00659,4.31,4.261897,4.021379,3.764828,3.508276,4.141415,0,0,0,0,0,0.0000000,0,1,0,0,1,0


In [18]:
dim (data_static_nn_1)

[1] 699  37

In [19]:
write.csv (data_static_nn_1, file = "processed_data/data_static_nn_1.csv")

## With embeddings

In [20]:
data_static_nn_2 = data_random_forest_statique %>%

    mutate (app.mthd = as.numeric (recode (app.mthd, "bc" = 0, "bsth" = 1, "os" = 2, "ts" = 3, "cs" = 4))) %>%

    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%

    mutate (incorp = as.numeric (recode (incorp, "none" = 0, "shallow" = 1, "deep" = 2)))

In [21]:
head (data_static_nn_2, n = 2) %>% embed

e.cum,e.rel,pmid,meas.tech,country,inst,ct,tan.app,app.rate,man.dm,man.ph,man.source,t.incorp,app.mthd,incorp,air_temp_1,air_temp_2,air_temp_3,air_temp_4,air_temp_5,air_temp_6,wind.2m_1,wind.2m_2,wind.2m_3,wind.2m_4,wind.2m_5,wind.2m_6,rain.rate_1,rain.rate_2,rain.rate_3,rain.rate_4,rain.rate_5,rain.rate_6
16.4190,0.1344607,182,micro met,DK,104,164.99,122.11,31.8,3.7,7.35,0,1000,0,0,8.20,7.648529,6.766177,5.883824,5.001471,9.38909,8.10,7.494118,6.524706,5.555294,4.585882,5.340230,0,0,0,0,0,0.0476788
6.9332,0.1188820,183,micro met,DK,104,163.20,58.32,21.6,2.8,7.71,0,1000,1,0,13.77,13.568793,12.562759,11.489655,10.416552,10.00659,4.31,4.261897,4.021379,3.764828,3.508276,4.141415,0,0,0,0,0,0.0000000


In [22]:
write.csv (data_static_nn_2, file = "processed_data/data_static_nn_2.csv")

# Data for recurrent neural networks

## Without embeddings

In [23]:
data_rnn_1 = data %>%

    mutate (app.mthd_bc = ifelse (app.mthd == "bc", 1, 0)) %>%
    mutate (app.mthd_bsth = ifelse (app.mthd == "bsth", 1, 0)) %>%
    mutate (app.mthd_ts = ifelse (app.mthd == "ts", 1, 0)) %>%
    mutate (app.mthd_os = ifelse (app.mthd == "os", 1, 0)) %>%

    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%

    mutate (incorp_none = ifelse (incorp == "none", 1, 0)) %>%
    mutate (incorp_shallow = ifelse (incorp == "shallow", 1, 0)) %>%

    select (- app.mthd, - incorp) %>% 

    mutate (e.cum_shift = c (0, e.cum [1 : (n() - 1)]), .by = pmid, .after = e.cum) %>%
    mutate (delta_e.cum = e.cum - e.cum_shift, .after = e.cum)

In [24]:
head (data_rnn_1, n = 2) %>% embed

e.cum,delta_e.cum,e.cum_shift,e.rel,j.NH3,pmid,meas.tech,country,inst,ct,dt,air.temp,wind.2m,rain.rate,tan.app,app.rate,man.dm,man.ph,man.source,t.incorp,app.mthd_bc,app.mthd_bsth,app.mthd_ts,app.mthd_os,incorp_none,incorp_shallow
7.1480,7.1480,0.000,0.0585374,1.7870,182,micro met,DK,104,4,4,8.20,8.10,0,122.11,31.8,3.7,7.35,0,1000,1,0,0,0,1,0
8.2921,1.1441,7.148,0.0679068,0.0673,182,micro met,DK,104,21,17,4.45,3.98,0,122.11,31.8,3.7,7.35,0,1000,1,0,0,0,1,0


In [25]:
dim (data_rnn_1)

[1] 6169   26

In [26]:
write.csv (data_rnn_1, file = "processed_data/data_rnn_1.csv")

In [27]:
data_rnn_1$pmid %>% unique %>% length

[1] 699

## With embeddings

In [28]:
data_rnn_2 = data %>%

    mutate (app.mthd = as.numeric (recode (app.mthd, "bc" = 0, "bsth" = 1, "os" = 2, "ts" = 3, "cs" = 4))) %>%

    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%

    mutate (incorp = as.numeric (recode (incorp, "none" = 0, "shallow" = 1, "deep" = 2))) %>% 

    mutate (e.cum_shift = c (0, e.cum [1 : (n() - 1)]), .by = pmid, .after = e.cum) %>%
    mutate (delta_e.cum = e.cum - e.cum_shift, .after = e.cum)

In [29]:
write.csv (data_rnn_2, file = "processed_data/data_rnn_2.csv")

## With embeddings and interpolation

In [30]:
data_random_forest_dynamique %>% head (n = 2)

,e.cum,e.cum_shift,dt,inst,pmid,country,meas.tech,ct,air.temp,wind.2m,rain.rate,tan.app,app.mthd,app.rate,man.dm,man.ph,man.source,incorp,t.incorp
,<dbl>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>
2,3.574,0.000,2,104,182,DK,micro met,2,8.2,8.1,0,122.11,bc,31.8,3.7,7.35,pig,none,1000
3,7.148,3.574,2,104,182,DK,micro met,4,8.2,8.1,0,122.11,bc,31.8,3.7,7.35,pig,none,1000


In [31]:
data_without_interpolation = data %>%

    mutate (app.mthd = as.numeric (recode (app.mthd, "bc" = 0, "bsth" = 1, "os" = 2, "ts" = 3, "cs" = 4))) %>%
    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%
    mutate (incorp = as.numeric (recode (incorp, "none" = 0, "shallow" = 1, "deep" = 2)))%>% 

    mutate (e.cum_shift = c (0, e.cum [1 : (n() - 1)]), .by = c (pmid), .after = e.cum) %>%
    mutate (delta_e.cum = e.cum - e.cum_shift, .after = e.cum) %>% 
    select (e.cum, delta_e.cum, e.cum_shift, dt, inst, pmid, country, meas.tech, ct, air.temp, wind.2m, rain.rate, 
            tan.app, app.mthd, app.rate, man.dm, man.ph, man.source, incorp, t.incorp)

In [32]:
data_rnn_3 = data_random_forest_dynamique %>%

    mutate (app.mthd = as.numeric (recode (app.mthd, "bc" = 0, "bsth" = 1, "os" = 2, "ts" = 3, "cs" = 4))) %>%
    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%
    mutate (incorp = as.numeric (recode (incorp, "none" = 0, "shallow" = 1, "deep" = 2))) %>% 

    mutate (e.cum_shift = c (0, e.cum [1 : (n() - 1)]), .by = c (pmid, dt), .after = e.cum) %>%
    mutate (delta_e.cum = e.cum - e.cum_shift, .after = e.cum)

In [33]:
data_rnn_3 = rbind (
    data_rnn_3 %>% mutate (interpolation = "yes"),
    data_without_interpolation %>% mutate (interpolation = "no")
)

In [34]:
# Adding the original time step (time step of the real data)
n = nrow (data_rnn_3)

data_rnn_3 = data_rnn_3 %>%
    mutate (dt_origin = NA, .after = dt)

for (i in c (1 : n)){
    ct_i = data_rnn_3$ct[i]
    pmid_i = data_rnn_3$pmid[i]

    dt_origin = data %>% filter ((pmid == pmid_i) & (ct >= ct_i)) %>% pull (dt) %>% head (1)

    # for the last ct_i of each pmid (last time for interpolation goes one step beyond last real time
    if ((dt_origin %>% length) == 1){dt_origin_saved = dt_origin}
    
    data_rnn_3$dt_origin[i] = ifelse ((dt_origin %>% length) == 0, dt_origin_saved, dt_origin)
}

In [35]:
data %>% filter (pmid == 182) %>% embed

e.cum,e.rel,j.NH3,pmid,meas.tech,country,inst,ct,dt,air.temp,wind.2m,rain.rate,tan.app,app.rate,man.dm,man.ph,man.source,t.incorp,app.mthd,incorp
7.1480,0.0585374,1.7870000,182,micro met,DK,104,4.00,4.00,8.20,8.10,0.0000000,122.11,31.8,3.7,7.35,pig,1000,bc,none
8.2921,0.0679068,0.0673000,182,micro met,DK,104,21.00,17.00,4.45,3.98,0.0000000,122.11,31.8,3.7,7.35,pig,1000,bc,none
11.8310,0.0968881,0.1490063,182,micro met,DK,104,44.75,23.75,7.22,6.57,0.0084211,122.11,31.8,3.7,7.35,pig,1000,bc,none
12.6320,0.1034477,0.0166045,182,micro met,DK,104,92.99,48.24,10.04,4.86,0.0559700,122.11,31.8,3.7,7.35,pig,1000,bc,none
16.4190,0.1344607,0.0525972,182,micro met,DK,104,164.99,72.00,12.13,5.38,0.0888890,122.11,31.8,3.7,7.35,pig,1000,bc,none


In [36]:
data_rnn_3 %>% head (n = 30) %>% embed

,e.cum,delta_e.cum,e.cum_shift,dt,dt_origin,inst,pmid,country,meas.tech,ct,air.temp,wind.2m,rain.rate,tan.app,app.mthd,app.rate,man.dm,man.ph,man.source,incorp,t.incorp,interpolation
2,3.574000,3.5740000,0.000000,2,4.00,104,182,DK,micro met,2,8.200000,8.100000,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
3,7.148000,3.5740000,3.574000,2,4.00,104,182,DK,micro met,4,8.200000,8.100000,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
4,7.282600,0.1346000,7.148000,2,17.00,104,182,DK,micro met,6,7.758824,7.615294,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
5,7.417200,0.1346000,7.282600,2,17.00,104,182,DK,micro met,8,7.317647,7.130588,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
6,7.551800,0.1346000,7.417200,2,17.00,104,182,DK,micro met,10,6.876471,6.645882,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
7,7.686400,0.1346000,7.551800,2,17.00,104,182,DK,micro met,12,6.435294,6.161176,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
8,7.821000,0.1346000,7.686400,2,17.00,104,182,DK,micro met,14,5.994118,5.676471,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
9,7.955600,0.1346000,7.821000,2,17.00,104,182,DK,micro met,16,5.552941,5.191765,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
10,8.090200,0.1346000,7.955600,2,17.00,104,182,DK,micro met,18,5.111765,4.707059,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes
11,8.224800,0.1346000,8.090200,2,17.00,104,182,DK,micro met,20,4.670588,4.222353,0.0000000,122.11,0,31.8,3.7,7.35,0,0,1000,yes


In [37]:
data_rnn_3 %>% count (interpolation)

interpolation,n
<chr>,<int>
no,6169
yes,92108


In [38]:
data_rnn_3 %>%
    filter (interpolation == "yes") %>% count (dt)

dt,n
<dbl>,<int>
2,39968
4,20144
6,13537
8,10219
10,8240


In [39]:
write.csv (data_rnn_3, file = "processed_data/data_rnn_3.csv")

In [40]:
data_rnn_3 %>% nrow

[1] 98277

# List of pmids for train, test and eval subsets (cross validation)

In [41]:
n_pmids = data$pmid %>% unique %>% length
n_test = floor (n_pmids * 0.1)
n_test

[1] 69

In [42]:
n_eval = n_test

In [43]:
# Calculating the proportion of trials by institute and the number of trials to sample for the test sets.

df_tmp_test = data %>%
    summarise (n_pmid = length (unique (pmid)), pmids = list (unique (pmid)), .by = "inst") %>%
    mutate (p = n_pmid / sum (n_pmid)) %>%
    mutate (sample_size = floor (p * n_test) + 1)

In [44]:
# Code check
df_tmp_test %>% head (2)
apply (df_tmp_test %>% head (2), 1, function (x) sample (x = x[[3]], x[[5]]))

,inst,n_pmid,pmids,p,sample_size
,<int>,<int>,<list>,<dbl>,<dbl>
1,104,28,"182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209",0.04005722,3
2,202,99,"910, 911, 913, 914, 916, 917, 918, 919, 920, 921, 922, 923, 924, 927, 928, 929, 930, 931, 932, 933, 934, 935, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 987, 988, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1016, 1017, 1018",0.14163090,10


$`1`
[1] 189 195 192

$`2`
 [1]  966  974  922  965 1010  933 1011  943  959  924

In [45]:
# Making 15 random partitions of the dataset into train, eval and test sets.

set.seed (123)

p = 15

list_test_pmids = list()
list_eval_pmids = list ()
list_train_pmids = list ()

for (i in c(1 : p)){
    test_pmids = apply (df_tmp_test, 1, function (x) sample (x = x[[3]], x[[5]])) %>% unlist %>% as.vector 
    list_test_pmids [[i]] = test_pmids

    df_tmp_eval = data %>%
        filter (!(pmid %in% test_pmids)) %>%
        summarise (n_pmid = length (unique (pmid)), pmids = list (unique (pmid)), .by = "inst") %>%
        mutate (p = n_pmid / sum (n_pmid)) %>%
        mutate (sample_size = floor (p * n_eval) + 1)
    
    eval_pmids = apply (df_tmp_eval, 1, function (x) sample (x = x[[3]], x[[5]])) %>% unlist %>% as.vector 
    list_eval_pmids [[i]] = eval_pmids

    train_pmids = setdiff (data %>% filter (!(pmid %in% test_pmids)) %>% pull (pmid) %>% unique, eval_pmids)
    list_train_pmids [[i]] = train_pmids

    c (test_pmids, eval_pmids, train_pmids) %>% unique %>% length %>% print

}      

[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699
[1] 699


In [46]:
save (list_test_pmids, file = "processed_data/list_test_pmids.Rdata")
save (list_eval_pmids, file = "processed_data/list_eval_pmids.Rdata")
save (list_train_pmids, file = "processed_data/list_train_pmids.Rdata")

write_json(list_test_pmids, "processed_data/list_test_pmids.json")
write_json(list_eval_pmids, "processed_data/list_eval_pmids.json")
write_json(list_train_pmids, "processed_data/list_train_pmids.json")

In [47]:
sapply (list_train_pmids, length)

[1] 550 550 550 550 550 550 550 550 550 550 550 550 550 550 550

In [48]:
sapply (list_eval_pmids, length)

[1] 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75

In [49]:
sapply (list_test_pmids, length)

[1] 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74

# List of pmids for final training set

In [50]:
n_pmids = data$pmid %>% unique %>% length
n_eval = 100

In [51]:
df_tmp_eval = data %>%
    summarise (n_pmid = length (unique (pmid)), pmids = list (unique (pmid)), .by = "inst") %>%
    mutate (p = n_pmid / sum (n_pmid)) %>%
    mutate (sample_size = floor (p * n_eval) + 1)

In [52]:
set.seed (123)

final_eval_pmids = apply (df_tmp_eval, 1, function (x) sample (x = x[[3]], x[[5]])) %>% unlist %>% as.vector 
final_train_pmids = setdiff (data$pmid %>% unique, final_eval_pmids)

In [53]:
length (final_eval_pmids)
length (final_train_pmids)

[1] 108

[1] 591

In [54]:
save (final_eval_pmids, file = "processed_data/final_eval_pmids.Rdata")
save (final_train_pmids, file = "processed_data/final_train_pmids.Rdata")

write_json(final_eval_pmids, "processed_data/final_eval_pmids.json")
write_json(final_train_pmids, "processed_data/final_train_pmids.json")

In [55]:
data %>% 
    filter (pmid %in% final_eval_pmids) %>%
    select (pmid, app.mthd, incorp, t.incorp) %>%
    distinct %>%
    count (app.mthd, incorp, t.incorp)  

app.mthd,incorp,t.incorp,n
<fct>,<fct>,<dbl>,<int>
bc,deep,0.0,1
bc,none,1000.0,23
bc,shallow,0.0,6
bc,shallow,1.5,1
bc,shallow,24.0,1
bsth,none,1000.0,33
cs,none,1000.0,2
os,none,1000.0,16
ts,none,1000.0,24


# Data for scenario predictions

In [56]:
# Same values as in Favrot et al. (2025)
temp_1 = c (rep (11.2, 4), rep (10.8, 4), rep (8.7, 4), rep (8.0, 4), rep (8.1, 4), rep (9.2, 52))
temp_2 = c (rep (18.2, 4), rep (18.5, 4), rep (16.2, 4), rep (14.2, 4), rep (14, 4), rep (16, 52))

wind_1 = c (rep (2.3, 4), rep (2.3, 4), rep (1.8, 4), rep (1.4, 4), rep (1.4, 4), rep (2.2, 52))
wind_2 = c (rep (5, 4), rep (4.6, 4), rep (3.8, 4), rep (3.7, 4), rep (3.7, 4), rep (4.1, 52))

rain_1 = c (rep (0, 4), rep (0, 4), rep (0, 4), rep (0, 4), rep (0, 4), rep (0, 52))
rain_2 = c (rep (0.3, 4), rep (0.2, 4), rep (0.3, 4), rep (0.4, 4), rep (0.3, 4), rep (0.1, 52))

In [57]:
t.incorp = c (0, 1000)
app.mthd = c ("bc", "bsth", "ts", "os", "cs")
incorp = c ("none", "shallow")

tan.app = c (36.7, 80.3)
app.rate = c (18.7, 36.7)
man.dm = c (3.8, 8.2)
man.ph = 7.5
man.source = c ("pig", "cat")

In [58]:
make_data_scenarios = function (time_seq){
    
    new_temp_1 = sapply (c(1 : (length (time_seq) - 1)), function (x) mean (temp_1[(time_seq[x] + 1) : time_seq[x+1]])) 
    new_temp_2 = sapply (c(1 : (length (time_seq) - 1)), function (x) mean (temp_2[(time_seq[x] + 1) : time_seq[x+1]])) 
    
    new_wind_1 = sapply (c(1 : (length (time_seq) - 1)), function (x) mean (wind_1[(time_seq[x] + 1) : time_seq[x+1]])) 
    new_wind_2 = sapply (c(1 : (length (time_seq) - 1)), function (x) mean (wind_2[(time_seq[x] + 1) : time_seq[x+1]]))
                         
    new_rain_1 = sapply (c(1 : (length (time_seq) - 1)), function (x) mean (rain_1[(time_seq[x] + 1) : time_seq[x+1]])) 
    new_rain_2 = sapply (c(1 : (length (time_seq) - 1)), function (x) mean (rain_2[(time_seq[x] + 1) : time_seq[x+1]])) 
    
    time_seq = time_seq [2 : length (time_seq)]
    
    df_temp = rbind (
        data_frame (air.temp = new_temp_1, ct = time_seq, group_temp = 1),
        data_frame (air.temp = new_temp_2, ct = time_seq, group_temp = 2)
    )
        
    df_wind = rbind (
        data_frame (wind.2m = new_wind_1, ct = time_seq, group_wind = 1),
        data_frame (wind.2m = new_wind_2, ct = time_seq, group_wind = 2)
    )
        
    df_rain = rbind (
        data_frame (rain.rate = new_rain_1, ct = time_seq, group_rain = 1),
        data_frame (rain.rate = new_rain_2, ct = time_seq, group_rain = 2)
    )
    
    group_temp = c (1, 2)
    group_wind = c (1, 2)
    group_rain = c (1, 2)
    
    hyper_grid_scenarios = expand_grid (group_temp, group_wind, group_rain, tan.app, app.rate, man.dm, man.ph, man.source) %>% 
        mutate (scenario = row_number(), .before = group_temp)
    
    
    hyper_grid_strategies = expand_grid (t.incorp, app.mthd, incorp) %>% 
        filter (! (incorp == "none" & t.incorp == 0)) %>%
        filter (! (incorp == "shallow" & t.incorp == 1000)) %>%
        filter (! (app.mthd %in% c ("bsth", "os", "ts", "cs") & incorp == "shallow")) %>%
        mutate (strategy = recode (app.mthd, 
                                   "bc" = "Broadcast",
                                   "bsth" = "Trailing hose",
                                   "ts" = "Trailing shoe",
                                   "os" = "Open slot",
                                   "cs" = "Closed slot")) %>%
        mutate (strategy = ifelse (app.mthd == "bc" & incorp == "shallow", "Incorporation", strategy)) %>%
        {.}
    
    hyper_grid = hyper_grid_scenarios %>%
        cross_join (hyper_grid_strategies) %>%
        mutate (pmid = row_number(), .after = scenario) %>%
        cross_join (data.frame (ct = time_seq)) %>%
        relocate (ct, .before = group_temp) %>%
        relocate (strategy, .before = ct)
    
    df_scenarios = hyper_grid %>%
        left_join (df_temp, by = c ("group_temp", "ct")) %>%
        left_join (df_rain, by = c ("group_rain", "ct")) %>%
        left_join (df_wind, by = c ("group_wind", "ct"))
    
    df_scenarios = df_scenarios %>%
        mutate_if (is.character, as.factor) %>%
        select (scenario, pmid, strategy, group_temp, group_wind, group_rain, ct, air.temp, wind.2m, rain.rate, tan.app, app.mthd, app.rate, man.dm, 
                man.ph, man.source, incorp, t.incorp)
}

## Code proof

In [59]:
ct = c (2, 4, 6)
groupe_temp = c (1, 2)
group_rain = c (1, 2)

hyper_grid = expand_grid (groupe_temp, group_rain) %>% 
    mutate (scenario = row_number(), .before = groupe_temp) %>%
    cross_join (data.frame (ct = ct)) %>%
    relocate (ct, .before = groupe_temp)

hyper_grid

scenario,ct,groupe_temp,group_rain
<int>,<dbl>,<dbl>,<dbl>
1,2,1,1
1,4,1,1
1,6,1,1
2,2,1,2
2,4,1,2
2,6,1,2
3,2,2,1
3,4,2,1
3,6,2,1


In [60]:
df_temp = rbind (
    data_frame (temp = c (11.2, 10.8, 8.7), ct = c (2, 4, 6), groupe_temp = 1),
    data_frame (temp = c (22, 22.2, 24), ct = c (2, 4, 6), groupe_temp = 2)
)
    
df_wind = rbind (
    data_frame (rain.rate = c (0, 0, 0), ct = c (2, 4, 6), group_rain = 1),
    data_frame (rain.rate = c (0.2, 0.3, 0.4), ct = c (2, 4, 6), group_rain = 2)
)

Warning message:
“`data_frame()` was deprecated in tibble 1.1.0.
ℹ Please use `tibble()` instead.”


In [61]:
hyper_grid %>%
    left_join (df_temp, by = c ("groupe_temp", "ct")) %>%
    left_join (df_wind, by = c ("group_rain", "ct"))

scenario,ct,groupe_temp,group_rain,temp,rain.rate
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2,1,1,11.2,0.0
1,4,1,1,10.8,0.0
1,6,1,1,8.7,0.0
2,2,1,2,11.2,0.2
2,4,1,2,10.8,0.3
2,6,1,2,8.7,0.4
3,2,2,1,22.0,0.0
3,4,2,1,22.2,0.0
3,6,2,1,24.0,0.0


## For dynamic random forest

In [62]:
scenarios_dynamic_rf = make_data_scenarios (time_seq = seq (0, 72, by = 2))

In [63]:
save (scenarios_dynamic_rf, file = "processed_data/scenarios_dynamic_rf.Rdata")

## For RNNs

In [64]:
c (0, seq (2, 72, by = 10))

[1]  0  2 12 22 32 42 52 62 72

In [65]:
scenarios_rnn1 = make_data_scenarios (time_seq = seq (0, 72, by = 2))
scenarios_rnn2 = make_data_scenarios (time_seq = seq (0, 72, by = 4))
scenarios_rnn3 = make_data_scenarios (time_seq = seq (0, 72, by = 6))
scenarios_rnn4 = make_data_scenarios (time_seq = seq (0, 72, by = 8))
scenarios_rnn5 = make_data_scenarios (time_seq = c (0, seq (2, 72, by = 10)))

In [66]:
scenarios_rnn2 %>% head 

scenario,pmid,strategy,group_temp,group_wind,group_rain,ct,air.temp,wind.2m,rain.rate,tan.app,app.mthd,app.rate,man.dm,man.ph,man.source,incorp,t.incorp
<int>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>
1,1,Incorporation,1,1,1,4,11.2,2.3,0,36.7,bc,18.7,3.8,7.5,pig,shallow,0
1,1,Incorporation,1,1,1,8,10.8,2.3,0,36.7,bc,18.7,3.8,7.5,pig,shallow,0
1,1,Incorporation,1,1,1,12,8.7,1.8,0,36.7,bc,18.7,3.8,7.5,pig,shallow,0
1,1,Incorporation,1,1,1,16,8.0,1.4,0,36.7,bc,18.7,3.8,7.5,pig,shallow,0
1,1,Incorporation,1,1,1,20,8.1,1.4,0,36.7,bc,18.7,3.8,7.5,pig,shallow,0
1,1,Incorporation,1,1,1,24,9.2,2.2,0,36.7,bc,18.7,3.8,7.5,pig,shallow,0


In [67]:
scenarios_rnn = rbind (
    
    scenarios_rnn1 %>% mutate (seq = 2),
    scenarios_rnn2 %>% mutate (seq = 4) %>% mutate (pmid = pmid + 1000),
    scenarios_rnn3 %>% mutate (seq = 6) %>% mutate (pmid = pmid + 2000),
    scenarios_rnn4 %>% mutate (seq = 8) %>% mutate (pmid = pmid + 3000),
    scenarios_rnn5 %>% mutate (seq = 10) %>% mutate (pmid = pmid + 4000)

) %>%

    mutate (app.mthd = as.numeric (recode (app.mthd, "bc" = 0, "bsth" = 1, "os" = 2, "ts" = 3, "cs" = 4))) %>%
    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%
    mutate (incorp = as.numeric (recode (incorp, "none" = 0, "shallow" = 1, "deep" = 2))) %>%

    mutate (ct_shift = c (0, ct[1 : (n() - 1)]), .by = c (pmid, seq)) %>%
    mutate (dt = ct - ct_shift, .after = ct)

In [68]:
write.csv (scenarios_rnn, file = "processed_data/scenarios_rnn.csv")

## For static models

In [69]:
static_scenarios = make_data_scenarios (time_seq = c (0, 4, 8, 12, 16, 20, 72))

In [70]:
static_scenarios = static_scenarios %>%
    rename (air_temp = air.temp) %>%
    mutate (group = c (1 : 6), .by = pmid, .before = ct) %>%
    mutate (ct = 72)

In [71]:
static_scenarios = static_scenarios %>%
    pivot_wider (names_from = group, values_from = c (air_temp, wind.2m, rain.rate))

In [72]:
static_scenarios %>% head %>% embed

scenario,pmid,strategy,group_temp,group_wind,group_rain,ct,tan.app,app.mthd,app.rate,man.dm,man.ph,man.source,incorp,t.incorp,air_temp_1,air_temp_2,air_temp_3,air_temp_4,air_temp_5,air_temp_6,wind.2m_1,wind.2m_2,wind.2m_3,wind.2m_4,wind.2m_5,wind.2m_6,rain.rate_1,rain.rate_2,rain.rate_3,rain.rate_4,rain.rate_5,rain.rate_6
1,1,Incorporation,1,1,1,72,36.7,bc,18.7,3.8,7.5,pig,shallow,0,11.2,10.8,8.7,8,8.1,9.2,2.3,2.3,1.8,1.4,1.4,2.2,0,0,0,0,0,0
1,2,Broadcast,1,1,1,72,36.7,bc,18.7,3.8,7.5,pig,none,1000,11.2,10.8,8.7,8,8.1,9.2,2.3,2.3,1.8,1.4,1.4,2.2,0,0,0,0,0,0
1,3,Trailing hose,1,1,1,72,36.7,bsth,18.7,3.8,7.5,pig,none,1000,11.2,10.8,8.7,8,8.1,9.2,2.3,2.3,1.8,1.4,1.4,2.2,0,0,0,0,0,0
1,4,Trailing shoe,1,1,1,72,36.7,ts,18.7,3.8,7.5,pig,none,1000,11.2,10.8,8.7,8,8.1,9.2,2.3,2.3,1.8,1.4,1.4,2.2,0,0,0,0,0,0
1,5,Open slot,1,1,1,72,36.7,os,18.7,3.8,7.5,pig,none,1000,11.2,10.8,8.7,8,8.1,9.2,2.3,2.3,1.8,1.4,1.4,2.2,0,0,0,0,0,0
1,6,Closed slot,1,1,1,72,36.7,cs,18.7,3.8,7.5,pig,none,1000,11.2,10.8,8.7,8,8.1,9.2,2.3,2.3,1.8,1.4,1.4,2.2,0,0,0,0,0,0


In [73]:
static_scenarios = static_scenarios %>%
    select (
        scenario, pmid, strategy, group_temp, group_wind, group_rain,
        ct, tan.app, app.rate, man.dm, man.ph, man.source, t.incorp, app.mthd, incorp, 
        air_temp_1, air_temp_2, air_temp_3, air_temp_4, air_temp_5, air_temp_6, 
        wind.2m_1, wind.2m_2, wind.2m_3, wind.2m_4, wind.2m_5, wind.2m_6, 
        rain.rate_1, rain.rate_2, rain.rate_3, rain.rate_4, rain.rate_5, rain.rate_6 
    )

In [74]:
scenarios_static_rf = static_scenarios

save (scenarios_static_rf, file = "processed_data/scenarios_static_rf.Rdata")

In [75]:
scenarios_static_nn = static_scenarios %>%

    mutate (app.mthd = as.numeric (recode (app.mthd, "bc" = 0, "bsth" = 1, "os" = 2, "ts" = 3, "cs" = 4))) %>%
    mutate (man.source = as.numeric (recode (man.source, "pig" = 0, "cat" = 1))) %>%
    mutate (incorp = as.numeric (recode (incorp, "none" = 0, "shallow" = 1, "deep" = 2)))


In [76]:
write.csv (scenarios_static_nn, file = "processed_data/scenarios_static_nn.csv")

# End